## 1. 写一个方法，实现从一个字串经过哪些操作后能得到另一个字串？

In [22]:
from functools import lru_cache
import copy

In [37]:
@lru_cache(maxsize=2**10) # 调用下面的函数时，若找到相同的输入，那么直接返回相同的输出结果
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2), []
    if len(string2) == 0: return len(string1), []
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    min_distance1, operations1 = edit_distance(string1[:-1], string2)
    min_distance1 += 1
    operations1 = copy.copy(operations1)
    operations1.append('DEL p{}_{}'.format(len(string1) - 1 , tail_s1),)  
    
    min_distance2, operations2 = edit_distance(string1, string2[:-1])
    operations2 = copy.copy(operations2)
    operations2.append('ADD p{}_{}'.format(len(string2) - 1 , tail_s2), )  
    min_distance2 += 1
    
    if tail_s1 == tail_s2:
        min_distance3, operations3 = edit_distance(string1[:-1], string2[:-1])
    else:
        min_distance3, operations3 = edit_distance(string1[:-1], string2[:-1])
        min_distance3 += 1
        operations3 = copy.copy(operations3)
        operations3.append('SUB p{}_{} => p{}_{}'.format(len(string1) - 1 ,tail_s1, len(string2) - 1, tail_s2))
    
    candidates = [
        (min_distance1, operations1),
        (min_distance2, operations2),
        (min_distance3, operations3),
    ] 
    
    min_distance, operations = min(candidates, key=lambda x: x[0])
    
    return min_distance, operations


In [38]:
edit_distance('ABCDECG','ABCCEF')

(3, ['SUB p3_D => p3_C', 'SUB p5_C => p5_F', 'DEL p6_G'])

In [39]:
edit_distance('特朗普称来自中国物资非常好','特朗普说中国物资都是垃圾')

(7,
 ['SUB p3_称 => p3_说',
  'DEL p4_来',
  'DEL p5_自',
  'SUB p10_非 => p8_都',
  'SUB p11_常 => p9_是',
  'SUB p12_好 => p10_垃',
  'ADD p11_圾'])

## 2. 如何让输入法输入字符串后在不带空格的时候完成修正？如何完成拼音的自动分割？

提示：使用第一节课提到的语言模型。

In [2]:
# import pinyin # 不知道为什么，安装后不能使用

先找个好用的拼音库：

In [ ]:
from xpinyin import Pinyin
pinyin = Pinyin()

In [11]:
pinyin.get_pinyin('我们是共产主义接班人')

# 默认以'-'为分割符
a = pinyin.get_pinyin(u"上海")
print(a)

# 默认以'-'分割,显示音调
b = pinyin.get_pinyin(u'上海', tone_marks='marks')
print(b)

# 默认以'-'分割,显示音调
b = pinyin.get_pinyin(u'上海', tone_marks='numbers')
print(b)

# 删除分割符
c = pinyin.get_pinyin(u'上海', '')
print(c)

# 设置空白格为分割符
d = pinyin.get_pinyin(u'上海', ' ')
print(d)

d1 = pinyin.get_initial(u'上')
print(d1)

d2 = pinyin.get_initials(u'上海')
print(d2)

d3 = pinyin.get_initials(u'上海', u'')
print(d3)

d4 = pinyin.get_initials(u'上海', u' ')
print(d4)

wordvalue = '上海'
s = pinyin.get_initials(wordvalue, u'').lower()
print(s)

# 多音字
print(pinyin.get_pinyin('重', tone_marks='numbers'))
print(pinyin.get_pinyin('和', tone_marks='numbers'))

shang-hai
shàng-hǎi
shang4-hai3
shanghai
shang hai
S
S-H
SH
S H
sh
zhong4
he2


In [1]:
from pypinyin import pinyin, lazy_pinyin, Style, load_phrases_dict, load_single_dict
from pypinyin.style import register

In [15]:
print(pinyin('你好'))  # [['nǐ'], ['hǎo']]
print(pinyin('中心中心', heteronym=True))  # 启用多音字模式  # [['zhōng', 'zhòng'], ['xīn']]
print(pinyin('中心中心', style=Style.FIRST_LETTER))  # 设置拼音风格，第一个字母 [['z'], ['x']]
print(pinyin('中心中心', style=Style.TONE2, heteronym=True))  # [['zho1ng', 'zho4ng'], ['xi1n']]
print(lazy_pinyin('中心'))  # 不考虑多音字的情况 # ['zhong', 'xin']

[['nǐ'], ['hǎo']]
[['zhōng', 'zhòng'], ['xīn'], ['zhōng', 'zhòng'], ['xīn']]
[['z'], ['x'], ['z'], ['x']]
[['zho1ng', 'zho4ng'], ['xi1n'], ['zho1ng', 'zho4ng'], ['xi1n']]
['zhong', 'xin']


In [18]:
##########处理不包含拼音的字符
# default (默认行为): 不做任何处理，原样返回:
print(lazy_pinyin('你好☆d☆你好'))  # ['ni', 'hao', '☆☆']
# ignore : 忽略该字符
print(lazy_pinyin('你好☆c☆你好', errors='ignore'))  # ['ni', 'hao']
# replace : 替换为去掉 \u 的 unicode 编码
print(lazy_pinyin('你好☆b☆你好', errors='replace'))  # ['ni', 'hao', '26062606']
# callable 对象 : 提供一个回调函数，接受无拼音字符(串)作为参数, 支持的返回值类型: unicode 或 list ([unicode, …]) 或 None 。
print(lazy_pinyin('你好☆a☆你好', errors=lambda x: 'star'))  # ['ni', 'hao', 'star']

########### 自定义拼音库
print(lazy_pinyin('还没', style=Style.TONE2))
load_phrases_dict({'桔子': [['jú'], ['zǐ']]})  # 增加 "桔子" 词组，可以自己定义
print(lazy_pinyin('桔子', style=Style.TONE2))

load_single_dict({ord('还'): 'hái,huán'})  # 调整 "还" 字的拼音顺序
print(lazy_pinyin('还没', style=Style.TONE2))

['ni', 'hao', '☆d☆', 'ni', 'hao']
['ni', 'hao', 'ni', 'hao']
['ni', 'hao', '2606622606', 'ni', 'hao']
['ni', 'hao', 'star', 'ni', 'hao']
['ha2i', 'me2i']
['ju2', 'zi3']
['ha2i', 'me2i']
['😘 me', '😘 me', '😘 dá']
['me', 'me', 'da']


In [ ]:
###########自定义拼音风格
@register('kiss')
def kiss(mypinyin, **kwargs):
    return '😘 {0}'.format(mypinyin)

print(lazy_pinyin('么么哒', style='kiss'))
print(lazy_pinyin('么么哒'))

还是pypinyin比较好，功能齐全！

接下来实现功能：

思路一： 训练时目标是构建一个字典，key是拼音，value是子字典（key是可能的词，value是 词库词频和用户输入次数等的数据对象），
训练时进行容错处理，应用时词选取根据词频和用户数据等信息计算后进行排序并显示给用户。

训练过程：

1. 用分词工具把语料切割成词，然后去重并统计每个词出现的次数。

2. 循环所有字和词的拼音以及容错性处理的拼音，建立这样一个字典。

3. 字典的持久化保存。

推理过程：

1. 如果用户输入的拼音能在字典中找到则直接计算出结果显示。

2. 如果用户输入的拼音太长则使用动态规划对拼音进行切割后执行1再拼接。

这一思路是本学生感觉最容易实现的一种方式。训练过程容易实现，容错容易处理，推理阶段计算实时高效，但内存可能消耗较大。